In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [31]:
df= pd.read_csv('data/EDANYRealEstate.csv')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35106 entries, 0 to 35105
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BOROUGH                  35106 non-null  object 
 1   NEIGHBORHOOD             35106 non-null  object 
 2   BUILDING CLASS CATEGORY  35106 non-null  object 
 3   BLOCK                    35106 non-null  int64  
 4   LOT                      35106 non-null  int64  
 5   ADDRESS                  35106 non-null  object 
 6   ZIP CODE                 35105 non-null  float64
 7   RESIDENTIAL UNITS        34292 non-null  float64
 8   COMMERCIAL UNITS         18928 non-null  float64
 9   TOTAL UNITS              35106 non-null  float64
 10  LAND SQUARE FEET         18114 non-null  float64
 11  GROSS SQUARE FEET        18114 non-null  float64
 12  YEAR BUILT               33071 non-null  float64
 13  TAX CLASS                35106 non-null  int64  
 14  BUILDING CLASS        

Since block and lot are specific to a plot, we will remove them as every borough has a separate block and lot.

In [33]:
df = df.drop(['BLOCK', 'LOT','ADDRESS'], axis = 1)

In [34]:
# we will remove data when yearbuilt is not available. 
df = df[df['YEAR BUILT'].isnull() == False]

In [35]:
df['TAX CLASS'] = df['TAX CLASS'].astype('category')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33071 entries, 0 to 35103
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   BOROUGH                  33071 non-null  object  
 1   NEIGHBORHOOD             33071 non-null  object  
 2   BUILDING CLASS CATEGORY  33071 non-null  object  
 3   ZIP CODE                 33071 non-null  float64 
 4   RESIDENTIAL UNITS        32433 non-null  float64 
 5   COMMERCIAL UNITS         18287 non-null  float64 
 6   TOTAL UNITS              33071 non-null  float64 
 7   LAND SQUARE FEET         17649 non-null  float64 
 8   GROSS SQUARE FEET        17649 non-null  float64 
 9   YEAR BUILT               33071 non-null  float64 
 10  TAX CLASS                33071 non-null  category
 11  BUILDING CLASS           33071 non-null  object  
 12  SALE PRICE               33071 non-null  int64   
dtypes: category(1), float64(7), int64(1), object(4)
memory usage:

In [37]:
# since Residential Unit and Total Units are correlated, we will remove RESIDENTIAL UNITS
# We will also remove COMMERCIAL UNITS as they are captured in Land Square Feet

df = df.drop(columns = ['RESIDENTIAL UNITS','COMMERCIAL UNITS' ])

In [38]:
for column in df.columns:
    print(column, ' : ' , df[column].nunique(), df[column].dtype)

BOROUGH  :  5 object
NEIGHBORHOOD  :  250 object
BUILDING CLASS CATEGORY  :  33 object
ZIP CODE  :  182 float64
TOTAL UNITS  :  45 float64
LAND SQUARE FEET  :  3181 float64
GROSS SQUARE FEET  :  2797 float64
YEAR BUILT  :  145 float64
TAX CLASS  :  3 category
BUILDING CLASS  :  112 object
SALE PRICE  :  4818 int64


In [39]:
df['ZIP CODE'] = df['ZIP CODE'].astype('category')

In [40]:
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'ZIP CODE',
       'TOTAL UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS', 'BUILDING CLASS', 'SALE PRICE'],
      dtype='object')

In [41]:
df = pd.get_dummies(df, columns=['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY','BUILDING CLASS','ZIP CODE', 'TAX CLASS'])

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33071 entries, 0 to 35103
Columns: 590 entries, TOTAL UNITS to TAX CLASS_4
dtypes: float64(4), int64(1), uint8(585)
memory usage: 20.0 MB


In [43]:
y = df['SALE PRICE'].values
X = df.iloc[:,df.columns!='SALE PRICE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

In [44]:
from sklearn.impute import KNNImputer
knn = KNNImputer(n_neighbors=5)
knn.fit(X_train)
X_train_impute = knn.transform(X_train)

In [45]:
X_test_impute = knn.transform(X_test)

In [50]:
pd.DataFrame(X_train_impute).to_csv('data/X_train.csv', index=False)
pd.DataFrame(X_test_impute).to_csv('data/X_test.csv', index=False)
pd.DataFrame(y_train).to_csv('data/y_train.csv', index=False)
pd.DataFrame(y_test).to_csv('data/y_test.csv', index=False)

In [51]:
scaler = StandardScaler()
scaler.fit(X_train_impute)
X_train_scaled = scaler.transform(X_train_impute)
X_test_scaled = scaler.transform(X_test_impute)

In [52]:
pd.DataFrame(X_train_scaled).to_csv('data/X_train_scaled.csv', index=False)
pd.DataFrame(X_test_scaled).to_csv('data/X_test_scaled.csv', index=False)
